# export @src  thành 1 cái API của chatbot chưa có UI 

To convert your existing chatbot code into an API without a UI, you can use a framework like Flask. Below are the necessary changes to create a simple API for your chatbot. This will allow you to send requests to the chatbot and receive responses without a user interface.

### Changes Overview
1. **Create a Flask app** to handle incoming requests.
2. **Define an endpoint** to receive user questions and return chatbot responses.
3. **Remove Streamlit-specific code** since we are not using a UI.

### Updated Code
Here’s how you can modify your `src/server.py` file:

```python:src/server.py
from flask import Flask, request, jsonify  # Import Flask
from rag_pipeline.back import LLMHandler, VectorDatabase, QuestionAnsweringChain
from dotenv import load_dotenv
import os

# Initialize Flask app
app = Flask(__name__)

# Parameters
load_dotenv()
gemini_key = os.getenv('gemini_key')
qdrant_key = os.getenv('qdrant_key')
rerank = True
rewrite = True
num_docs = 5

# Initialize components
vector_db = VectorDatabase(
    model_name="hiieu/halong_embedding",
    collection_name='cmc_final_db',
    api=qdrant_key
)
llm_handler = LLMHandler(model_name="gemini-1.5-flash", gemini_key=gemini_key)
qa_chain = QuestionAnsweringChain(
    llm_handler=llm_handler,
    vector_db=vector_db,
    num_docs=num_docs,
    apply_rerank=rerank,
    apply_rewrite=rewrite,
    date_impact=0.001
)

@app.route('/chat', methods=['POST'])  # Define the chat endpoint
def chat():
    data = request.json
    question = data.get('question')
    
    if not question:
        return jsonify({"error": "No question provided"}), 400

    # Get AI response
    response, extracted_links = qa_chain.run(question)

    return jsonify({
        "response": response,
        "links": extracted_links
    })

if __name__ == '__main__':
    app.run(debug=True)  # Run the Flask app
```

### Explanation of Changes
- **Flask Setup**: The code imports Flask and initializes a Flask app.
- **Chat Endpoint**: A new endpoint `/chat` is created to handle POST requests. It expects a JSON body with a `question` field.
- **Response Handling**: The chatbot's response and any extracted links are returned as a JSON response.
- **Removed Streamlit Code**: All Streamlit-specific code has been removed since we are now using an API.

### Running the API
To run the API, make sure you have Flask installed. You can install it using:
```bash
pip install Flask
```
Then, run your server:
```bash
python src/server.py
```

You can now send POST requests to `http://127.0.0.1:5000/chat` with a JSON body like:
```json
{
    "question": "Your question here"
}
```

This will return a JSON response with the chatbot's answer and any related links.
